In [20]:
# Install the google-genai SDK for this codelab.
#!pip install -qU 'google-genai==1.7.0'

In [21]:
from google import genai
from google.genai import types
import os
from IPython.display import Markdown, HTML, display

genai.__version__

'1.7.0'

In [22]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, "__wrapped__"):
    genai.models.Models.generate_content = retry.Retry(predicate=is_retriable)(
        genai.models.Models.generate_content
    )

In [ ]:
GOOGLE_API_KEY = "Your Google API Key"

In [24]:
from IPython.display import display, Image, Markdown


def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())

        display(Markdown("----"))

In [25]:
class GeminiCodingAgent:
    def __init__(self, api_key=None, model_name="gemini-2.0-flash"):
        """
        Initializes the GeminiCodingAgent with a Google Gemini model.
        Ensure you have a Google Cloud API key set as an environment variable
        GOOGLE_API_KEY or provide it during initialization.
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GOOGLE_API_KEY")
        if not self.api_key:
            raise ValueError(
                "Google API key is required. Set the GOOGLE_API_KEY environment variable or provide it during initialization."
            )

        # genai.configure(api_key=self.api_key)

        try:
            self.client = genai.Client(api_key=GOOGLE_API_KEY)
            # self.model = genai.GenerativeModel(self.model_name)
            print(f"Successfully initialized Gemini model: {self.model_name}")
        except Exception as e:
            print(f"Error initializing Gemini model {self.model_name}: {e}")
            self.client = None

    def generate_code(
        self, user_prompt, max_output_tokens=1000, num_return_sequences=1
    ):
        """
        Generates Python code using the Gemini model based on the user's prompt.

        Args:
            user_prompt (str): The natural language description of the desired code.
            max_output_tokens (int): The maximum number of tokens in the generated code.
            num_return_sequences (int): The number of code snippets to generate (Gemini might handle this differently).

        Returns:
            ressponse: res or None if the model is not initialized.
        """
        config_with_search = types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())],
            max_output_tokens=max_output_tokens,
            candidate_count=num_return_sequences,
            # system_instruction=self.system_prompt(),
        )

        if self.client:
            try:
                response = self.client.models.generate_content(
                    model=self.model_name,
                    contents=user_prompt,
                    config=config_with_search,
                )
                return response
                # return [part.text for part in response.candidates[0].content.parts if hasattr(part, "text")]
            except Exception as e:
                print(f"Error during code generation with Gemini: {e}")
                return None
        else:
            print("Gemini model not initialized.")
            return None

    def system_prompt(self):
        return """You are a helpful chatbot that can answers all question related to coding. 
        You will take the users questions and return answer with minimum words possible.
        queries using the tools available. Once you have the information you need, you will
        answer the user's question using the data returned.
        you can use google search tool for any latest information from the internet
        or if user asking for google search.
        """

    def interact(self):
        """
        Basic interaction loop with the Gemini coding agent.
        """
        display(Markdown("----" * 20))
        display(Markdown("## Welcome to the Gemini-powered Coding Assistant!"))
        while True:
            user_input = input("Describe the Python code you need (or type 'exit'): ")
            if user_input.lower() == "exit":
                break

            if self.client:
                print("Generating code using Gemini...")
                print(f"Q. {user_input}\n")
                response = self.generate_code(user_input)
                if response:
                    show_response(response)
                else:
                    print("Failed to generate code with Gemini.")
            else:
                print("Gemini model is not available.")

In [26]:
agent = GeminiCodingAgent(GOOGLE_API_KEY, model_name="gemini-2.0-flash")
agent.interact()

Successfully initialized Gemini model: gemini-2.0-flash


--------------------------------------------------------------------------------

## Welcome to the Gemini-powered Coding Assistant!

Generating code using Gemini...
Q. What is the pandas function to check null values?



The primary function in pandas to check for null values is `isnull()`.

Here's how you can use it:

1.  **To check for null values in a DataFrame:**

    *   `df.isnull()`: This will return a DataFrame of the same shape as `df`, but with boolean values. Each `True` indicates a null value, and each `False` indicates a non-null value.
    *   `df.isnull().sum()`: This counts the number of null values in each column of the DataFrame.
2.  **To check for null values in a Series:**

    *   `series.isnull()`:  This returns a Series with boolean values, where `True` indicates a null value and `False` indicates a non-null value.
3.  **To check if a particular cell is null:**

    *   `df.isnull().iloc[row_index, column_index]`:  This returns `True` if the cell at the specified row and column index is null, and `False` otherwise.

In addition to `isnull()`, there's also a `notnull()` function, which does the opposite. It returns `True` for non-null values and `False` for null values.


...

In Python, a `property` is a way to manage class attributes, providing more control over how they are accessed and modified. It's a powerful feature that allows you to encapsulate attribute access and add logic to the process, like validation or computation, without changing the class's public interface.

Here's a breakdown of what `property` in Python entails:

**1. Managed Attributes**

*   `property` turns class attributes into what are often called "managed attributes".
*   It provides a way to define methods that can be accessed like attributes, offering intuitive access to a class's data.

**2. Getters, Setters, and Deleters**

*   The `property` mechanism allows you to define getter, setter, and deleter methods for an attribute.
    *   **Getter:**  A method to retrieve the value of the attribute.
    *   **Setter:** A method to set or modify the value of the attribute, allowing you to add validation or processing logic.
    *   **Deleter:** A method to delete the attribute.

**3. How to use `property`**

*   **`property()` function:** `property()` is a built-in function in Python used to create a property of a class
*   **`@property` decorator:**  The most common and Pythonic way to define a property is by using the `@property` decorator.

**4. Benefits of Using `property`**

*   **Encapsulation:**  Hides the internal implementation details of an attribute and prevents direct access, thus protecting data integrity.
*   **Data Validation:** Allows you to validate new values before setting them to an attribute, preventing incorrect or invalid data.
*   **Readability:** Makes code cleaner and more readable by allowing you to access methods like attributes
*   **Computed Attributes:** Can be used to create attributes whose values are computed dynamically.
*   **Backward Compatibility:**  Allows you to modify the internal implementation of a class without affecting the external code that uses it.
*   **Controlled Access:**  You can make attributes read-only (by only defining a getter) or provide controlled read-write access with validation.


----

Generating code using Gemini...
Q. What is latest version of python released?



The latest stable version of Python available for download is **Python 3.13.3**, which was released on April 8, 2025. However, it's worth noting that Python 3.13 is a recent major version, officially released on October 7, 2024.


----